In [1]:


import numpy as np
import networkx as nx
import os
import pandas as pd
import torch
import time
import math
import dgl as dgl
from dgl.nn import pytorch
import numpy as np
import torch as th
import dgl.function as fn
import torch.nn.functional as F
from dgl.data import CoraGraphDataset
from GraphSAGE.losses import compute_loss_multiclass
from utils import *
from model import *
from loss import *
import matplotlib.cm as cm
import matplotlib.pyplot as plt



def train(g, features, n_classes, in_feats, n_edges, labels, mask, Q, cuda, nn_model,loss_direction):
    # sethyperparameter
    dropout = 0.0
    gpu = 0
    lr = 0.0005
    early_stop_rate=0.000005
    n_epochs = 1000
    n_hidden = features.shape[1]  # number of hidden nodes
    n_layers = 1  # number of hidden layers
    weight_decay_gamma = 0.65 #
    self_loop = True  #
    early_stop = False
    visualize_model=False
    last_score = 0
    step_size=int(n_epochs/100)

    # step_size = 1
    if self_loop:
        g = dgl.add_self_loop(g)
    # run single train of some model
    degs = g.in_degrees().float()
    norm = torch.pow(degs, -0.5)
    norm[torch.isinf(norm)] = 0

    if cuda:
        torch.cuda.set_device(gpu)
        features = features.cuda()
        labels = labels.cuda()
        g = g.to('cuda:0')
        mask = mask.cuda()

    if cuda:
        norm = norm.cuda()
    # g.ndata['norm'] = norm.unsqueeze(1)

    if nn_model == 'GCN':
        model = eval(nn_model)(g,
                               in_feats,
                               n_hidden,
                               n_classes,
                               n_layers,
                               F.relu,
                               dropout)
        if cuda:
            model.cuda()
    else:
        model = eval(nn_model)(in_feat=in_feats,
                               out_feat=n_classes,
                               num_rels=3,
                               regularizer='basis',
                               num_bases=None,
                               bias=True,
                               activation=None,
                               self_loop=True,
                               low_mem=False,
                               dropout=0.0,
                               layer_norm=False
                               )
        if cuda:
            model.cuda()

    loss_fcn = ModularityScore(n_classes, cuda,loss_direction)


    if visualize_model:
        print_parameter(model)
        print_parameter(loss_fcn)

    #optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    #apply weight_decay scheduler
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    StepLR = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=weight_decay_gamma)

    # train and evaluate (with modularity score and labels)
    dur = []



    for epoch in range(n_epochs):
        model.train()
        t0 = time.time()
        C_hat = model(features)
        # use train_mask to train
        loss = loss_fcn(C_hat[mask], Q)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        StepLR.step()

        dur.append(time.time() - t0)
        if epoch % step_size == 0:
        #if epoch % 1 == 0:
            if visualize_model:
                print_parameter(model)
                print_parameter(loss_fcn)
            C_out,eval_loss = evaluate_M(C_hat,Q,cuda)
            print("Epoch {} | Time(s) {}  Train_Modularity {} | True_Modularity {}"
                  "ETputs(KTEPS) {}".format(epoch, np.mean(dur), eval_loss, abs(loss),
                                             n_edges / np.mean(dur) / 1000))
        if early_stop:
            if abs((loss - last_score) / last_score) < 0.000005 or torch.isnan(loss).sum()>0:
                loss=last_score
                C_hat=last_C_hat
                C_out, eval_loss = evaluate_M(C_hat, Q, cuda)
                print("Epoch {} | Time(s) {}  Train_Modularity {} | True_Modularity {}"
                      "ETputs(KTEPS) {}".format(epoch, np.mean(dur), eval_loss, abs(loss),
                                                n_edges / np.mean(dur) / 1000))
                break

        last_score = loss
        last_C_hat=C_hat

    #C_out = C_construction(model, features, mask)

    C_init,modularity_init = evaluate_M(features, Q, cuda)
    print('initial modularity is', modularity_init)
    C_hat,modularity_score = evaluate_M(C_hat, Q, cuda)
    if torch.isnan(modularity_score):
        modularity_score=loss
    print(C_hat)
    return modularity_score.cpu().detach().numpy(), C_hat.cpu(), model.__str__(),features.cpu()


def main(nx_g, nn_model,loss_direction):
    # note g is a networkx class
    gpu = 0
    if gpu < 0:
        cuda = False
    else:
        cuda = True
    # prepare training data, set hyperparameters
    g, features, n_classes, in_feats, n_edges, labels,Q,mask,modularity_classic = generate_model_input(nx_g,cuda)
    return train(g, features, n_classes, in_feats, n_edges, labels, mask, Q, cuda, nn_model,loss_direction),modularity_classic




Using backend: pytorch
C:\Users\benno\anaconda3\envs\gpu\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:


if __name__ == "__main__":
    test_number = 10
    work_dir = os.getcwd()
    nn_model = 'GCN'
    data_dir = os.path.join(work_dir, 'data/ComboSampleData/')
    # G = loadNetworkMat('karate_34.mat',data_dir)
    G = loadNetworkMat('celeganmetabolic_453.mat', data_dir)
    modularity_scores_gcn = {}
    nmi_gcn={}
    nmi={}
    C_init={}
    C_out = {}
    C_out_combo = {}
    graph_type = {}
    modularity_scores_combo = {}
    model_parameter = {}
    data_name = []
    modularity_scores_classic={}
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file[-3:] == 'mat':
                if file !='email_1133.mat':
                    continue
                data_name.append(file)
                G = loadNetworkMat(file, data_dir)
                if nx.classes.function.is_directed(G):
                    graph_type[file] = 'directed'
                else:
                    graph_type[file] = 'undirected'
                print(file, graph_type[file])

                # need to figure out it is weighted or not
                modularity_scores_combo[file], partition = getNewComboPartition(G)
                C_out_combo[file] = partition_to_binary_attachment(partition)
                [modularity_scores_gcn[file], C_out[file], model_parameter[file],C_init[file]],modularity_scores_classic[file] = main(G, nn_model,loss_direction=1)

                nmi_gcn[file]=NMI(C_out[file],C_init[file])
                nmi[file] = NMI(C_out_combo[file], C_init[file])



    ##save log
    save_result(data_name, graph_type, modularity_scores_gcn, modularity_scores_combo, modularity_scores_classic,nmi_gcn,nmi,model_parameter,
                data_dir)

    print('something')

email_1133.mat undirected


C:\Users\benno\anaconda3\envs\gpu\lib\site-packages\dgl\base.py:45: DGLWarning: DGLGraph.__len__ is deprecated.Please directly call DGLGraph.number_of_nodes.
  return warnings.warn(message, category=category, stacklevel=1)


Epoch 0 | Time(s) 0.057012081146240234  Train_Modularity 0.5711579322814941 | True_Modularity 0.4084444046020508ETputs(KTEPS) 191.22262827128796
Epoch 10 | Time(s) 0.012820959091186523  Train_Modularity 0.5395971536636353 | True_Modularity 0.35311007499694824ETputs(KTEPS) 850.3264008926081
Epoch 20 | Time(s) 0.010335513523646764  Train_Modularity 0.5142191648483276 | True_Modularity 0.3184286952018738ETputs(KTEPS) 1054.809707815404
Epoch 30 | Time(s) 0.009647338621077998  Train_Modularity 0.5007920265197754 | True_Modularity 0.29947757720947266ETputs(KTEPS) 1130.0525904813535
Epoch 40 | Time(s) 0.00956309132459687  Train_Modularity 0.4719623327255249 | True_Modularity 0.28619131445884705ETputs(KTEPS) 1140.0079357141944
Epoch 50 | Time(s) 0.009139261993707395  Train_Modularity 0.46494364738464355 | True_Modularity 0.2762044668197632ETputs(KTEPS) 1192.875311759997
Epoch 60 | Time(s) 0.00898560148770692  Train_Modularity 0.4592546224594116 | True_Modularity 0.26862484216690063ETputs(KTEPS

In [3]:

if __name__ == "__main__":
    test_number = 10
    work_dir = os.getcwd()
    nn_model = 'GCN'
    data_dir = os.path.join(work_dir, 'data/ComboSampleData/')
    # G = loadNetworkMat('karate_34.mat',data_dir)
    G = loadNetworkMat('celeganmetabolic_453.mat', data_dir)
    modularity_scores_gcn = {}
    nmi_gcn={}
    nmi={}
    C_init={}
    C_out = {}
    C_out_combo = {}
    graph_type = {}
    modularity_scores_combo = {}
    model_parameter = {}
    data_name = []
    modularity_scores_classic={}
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file[-3:] == 'mat':
                if file !='email_1133.mat':
                    continue
                data_name.append(file)
                G = loadNetworkMat(file, data_dir)
                if nx.classes.function.is_directed(G):
                    graph_type[file] = 'directed'
                else:
                    graph_type[file] = 'undirected'
                print(file, graph_type[file])

                # need to figure out it is weighted or not
                modularity_scores_combo[file], partition = getNewComboPartition(G)
                C_out_combo[file] = partition_to_binary_attachment(partition)
                [modularity_scores_gcn[file], C_out[file], model_parameter[file],C_init[file]],modularity_scores_classic[file] = main(G, nn_model,loss_direction=-1)

                nmi_gcn[file]=NMI(C_out[file],C_init[file])
                nmi[file] = NMI(C_out_combo[file], C_init[file])



    ##save log
    save_result(data_name, graph_type, modularity_scores_gcn, modularity_scores_combo, modularity_scores_classic,nmi_gcn,nmi,model_parameter,
                data_dir)

    print('something')




email_1133.mat undirected


C:\Users\benno\anaconda3\envs\gpu\lib\site-packages\dgl\base.py:45: DGLWarning: DGLGraph.__len__ is deprecated.Please directly call DGLGraph.number_of_nodes.
  return warnings.warn(message, category=category, stacklevel=1)


Epoch 0 | Time(s) 0.013002634048461914  Train_Modularity 0.5673825144767761 | True_Modularity 0.4041607975959778ETputs(KTEPS) 838.4454995324276
Epoch 10 | Time(s) 0.010365919633345171  Train_Modularity 0.5673825144767761 | True_Modularity 0.3924739956855774ETputs(KTEPS) 1051.7156591616206
Epoch 20 | Time(s) 0.010907207216535295  Train_Modularity 0.5673825144767761 | True_Modularity 0.3874433636665344ETputs(KTEPS) 999.5225893822388
Epoch 30 | Time(s) 0.010260335860713836  Train_Modularity 0.5673825144767761 | True_Modularity 0.3846389949321747ETputs(KTEPS) 1062.5383172633806
Epoch 40 | Time(s) 0.00956307969442228  Train_Modularity 0.5673825144767761 | True_Modularity 0.3829936385154724ETputs(KTEPS) 1140.0093221390441
Epoch 50 | Time(s) 0.009256923899931066  Train_Modularity 0.5673825144767761 | True_Modularity 0.3820142447948456ETputs(KTEPS) 1177.7130413788088
Epoch 60 | Time(s) 0.009116696529701108  Train_Modularity 0.5673825144767761 | True_Modularity 0.38139212131500244ETputs(KTEPS) 